https://gist.github.com/saprativa/b5cb639e0c035876e0dd3c46e5a380fd

In [20]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [23]:
# open and read the file from google drive
file = open("transcript.txt", "r")
FileContent = file.read().strip()

In [ ]:
# total characters in the file
len(FileContent)

39723

In [ ]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
# max tokens including the special tokens
tokenizer.model_max_length

1024

In [ ]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [ ]:
# number of special tokens
tokenizer.num_special_tokens_to_add()

2

## Convert file content to sentences

In [ ]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /home/rohithnd/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

93

In [ ]:
# create chunks 


# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

10

In [ ]:
# some checks
[len(tokenizer.tokenize(c)) for c in chunks]

[1014, 984, 960, 1005, 1016, 994, 991, 1005, 1017, 576]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[1016, 986, 962, 1007, 1018, 996, 993, 1007, 1019, 578]

In [ ]:
# with spl tokens 
sum([len(tokenizer(c).input_ids) for c in chunks])

9582

In [ ]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (9564 > 1024). Running this sequence through the model will result in indexing errors


9564

In [ ]:
# without special tokens
sum([len(tokenizer.tokenize(c)) for c in chunks])

9562

In [ ]:
len(tokenizer.tokenize(FileContent))

9562

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Market research shows that TV remote control has a fancy look and feel, not a functional look or or feel, the number one thing that was found was that television remote control was not functional. Number two was that it be innovative without a adding unnecessary functional bits to it, and third priority is that it has to be user friendly while still having technology.
 Style is number one thing in the in the market of who we're selling to. Innovative design technology's also a must in that it's seen it'd be seen to be uh cutting edge, but ease of use t has to be insured throughout. And then at the end there are vibrant natural colours.
 We need to have something that unifies a lot of the different concepts, and if we think that what we are w our number one marketing motive is the look and feel. We are leaning quite a bit to the side of being low-tech, rubber buttons plastic frame, it's almost like we're reproducing the remote control that's out there. We're gonna need to put in a real